# Predicting Checkins.


Here I will summarise my best submission(0.58685(~116pub)-0.58637(~130pri) based on xgboost), but before that I will try to explain my various approaches to the problem and my learnings during this competition.

Here is my code realated to all my approaches. https://github.com/syllogismos/facebook-v-predicting-check-ins

This is my first proper shot at a kaggle competition I participated in. I participated in few earlier some 2/3 years back.. but didnt do much, just put the data in some algorithm without really tuning the parameters and etc..

I learned a lot from the forums and how to approach the problems. Some validated my approaches and some taught me about how different classifiers work and how to tune them.

### I liked two posts specifically.
    1. The post about how just submitting the top 3 place ids in a grid cell. This is a really nice intuition. My first intuition is to calculate the median x,y of all the place ids and for each checkin event from test set submit the closest. The top3 approach had a better score than mine. And also made me realize the importance of starting simple and then iterating on our approaches. I also participated one of the earlier facebook competitions facebook link prediction. Given the friend network you have to predict the new friend connections that are most likely to happen. which is called link prediction. Instead of starting simple I started implementing a paper. I understood the math in it and all that, but in the end I wasn't even able to generate a submission using my implementation.  I wrote about the implementation here http://syllogismos.github.io/blog/2014/08/10/facebook-link-prediction/

    2. How the time features like hour of the day and etc are converted into periodical features using sin and cos. This didn't really help me much. But made me realize how to go about feature engineering.. for future kaggle competitions :D

## My approaches:
    1. Initially I divided the training data based on a random split. People in forums suggestd splitting it based on some time. All events after a point of time are cv and before are train. 
    2. Find the median/mean/mode of all place ids. and then for each checkin event from the test set, find the closest three places. I tried both eucledian and manhattan distances.
    3. I also implimented the top 3 places in a each grid cell for all the points falling in the grid.
    4. Next I divided the world into mxn square grids with some buffer on both sides and only trained on placeids that came atleast some n number of times. Most of the people in the forums were also doing the same which validated what I wanted to do also. So I made a general frame work where I can put in any classifier in a single grid. I also parallalized how the training happens. Say the world is divided into mxn grids. I would be processing each row parallally. I also used the multiple threads the classifier itself provides. For example, in case of xgboost, I would have nthreads = 4, and would process 8 rows at a time in c4.8xlarge machine. My best submission took around 3.5 hours to train and generate the submission which had 40 rows and 200 columns.

    Using this framework I tried, KNeighbours, GaussianNB, and xgboost. I wasn't doing really well on the leader board, was around 600 or something with 10 days to go. At the least I wanted to beat the best open script submission out there. I gave up all hope and then suddenly when I reduced the grid size in my xgboost implementation I fell in top 100 with 6 days to go. Which felt really good.  I tried various ways of improving my lb score. I had three things in mind. 
        * Instead of using the same params for all the grids, why not do a grid cv search in each cell. Say for my best performing grid till then. There are 40*200 grids, That is 8000 diff grid cv searches... Instead of doing a grid search for all grids, I did a grid search for m, n grid.. and used those params for all the grids that fall into a bigger grid. I implemented this and didn't really find much improvement. And the gridcv search took almost a day to finish as far as I can remember. Not only did it take so much time to implement and finally be able to use those params. It didn't even improve my score. Which was a bummer.
        * The other thing I have in my mind was to use the top 10 predictions I did on the training/test data to with one run and some how extract even more features based on the top 10 predicted place ids. All I could think of was to make use of the cardinality of the top 10 predicted place ids, median of the accuracy of those ids, median of x, y.. distance of the check in event from the top 10 places.
        Ex: for event x, y, a, t my predictions are p1, p2, p3, p4.. p10 using a classifie clf1
        I would generate more features from each of these prdiction p1.. p10
        like median of the accuracy of all the p1 events from train data. median position of p1, no of occurances of p1 in training data.. distance of the check in event from median p1. for all the top 10 predictions..
        Initially I wanted to try neural nets or some other classification algorithm in my grid framework. But instead I convinced myself, more feature engineering is better than trying a diff model and went ahead with implementing this instead. So now instead of just x, y, a, and time features, I also have all the place id characterestics to predict. After spending considerable amount of time on this. It brought some interesting problems my way also when it came to impelemntation. The training data suddely is around 30 GB now. It did well on my cross validation set but it also didnt improve my lb score. I think because of the total no of features I have now, it is over fitting.. Bummerrr..
        * One more thing I had in my mind is that.. out of all the test events.. If I were to somehow be able to identify all the checkin events where the predictions with the second place as the right palce id. As in if my earlier classifier predicted p1, p2, p3 s the predictions.. I have the top n prediction data for the training and cv data also.. say if p2 is sort of the place that is more likely to not be first place.. and if I were to successfully identify that, my score would increase.. I did a validation of this.. So for my cv, about 59% of my predictions are where the p1 the first prediction is the right check in place. and 13% p2, and so on.. And if I were to identify those 13% checkin events successfully.. My score would increase.. But this also didn't work.
        
        After trying and failing to come up with more features, I'm soo interested in how people who are top in lb got there. Now that I think about it, instead of trying all these hypothesis on getting more features and feature engineering, I should have instead tried a different algorithm :D
        In the end I was only able to improve my score slightly. By trying different size grids. and different params and diff thresholds of cardinality of a place.
        
        OK enough of my essays.. lets get to the actual implementaion and code.

In [ ]:
# here is the repo, https://github.com/syllogismos/facebook-v-predicting-check-ins
# cd facebook-v-predicting-check-ins
# have train.csv, cv.csv, test.csv in this folder
# jupyter notebook and open this notebook
cd scripts/

In [ ]:
# scripts/grid_generation.py
from grid_generation import Grid

# grid defn: x, y, xd, yd
# and assume 2*xd < x, 2*yd < y.. don't know if this is necessary.. but for now
# im lazy to evaluate
# x is the length wise and y is the height wise grid length. xd and yd are the
# buffers added on both sides of x and y.
# 
#            ....xxxxxxxxxxxxxxxxxxxx....
#            .                          .
#            y   xxxxxxxxxxxxxxxxxxxx   y
#            y   y                  y   y
#         y  y   y                  y   y
#            y   xxxxxxxxxxxxxxxxxxxx   y
#        yd  .                         .
#            ....xxxxxxxxxxxxxxxxxxxx....
#             xd          x
# 
# every individual world has all the training data inside the outer rectangle that includes
# the buffer.
# when a test data point is considered, you select the world in which the test point falls in
# the inner rectangle

g = Grid(250, 50, 20, 5, pref = 'grid', files_flag = True, train_file = '../train.csv')
# 250, 50, 20, 5 are all in meters.
# it means a single grid cell is 250 mts wide, 50 mts height, 20 mts buffer along x axis and 5 mts buffer on y axis
# so m = 10000/250 and n = 10000/50 a 40X200

g.generateCardinalityMatrix()
# this will generate all train data into respective grid files, and also calculates the cardinality matrix of all
# place ids in respective grid. if files_flag above is True, it will generate 40*200 files. You might have to increase 
# your ulimit settings of os. don't recommend in local machine but in aws pls go wild
# for bigger grid cells you wont have a problem even in local machine.
# this also generates a cardinality matrix where g.M[i][j] will have a dictionary of all the place ids and its count 
# in grid i, j
# I wanted to do this computation only once per grid defn, so this is why i did it. I also pickle the cardinality matrix
# and save it along with all the individual grid train files in the folder ../grid_250_50_20_5
# if you are running generateCardinalityMatrix for the first time, it will create all the files necessary during its first
# run only, and keep using those files in later runs as long as you didn't chage the grid defn

In [ ]:
# scripts xgboost_time_features.py
# you can do a sample xgboost run like this
import xgboost_time_features as xgbm


xgbm1 = xgbm.XGB_Model(grid = g, threshold = 7, cross_validation_file='../cv.csv')
# grid is the above grid object, threshold is the threshold of the cardinality of a place for it to be considered
# while training, and cross validation file

params1 = {
        'objective': 'multi:softprob',
        'eta': 0.1,
        'max_depth': 5,
        'min_child_weight': 4,
        'gamma': 0.3,
        'subsample': 0.9,
        'colsample_bytree': 0.7,
        'colsample_bylevel': 0.9,
        'scale_pos_weight': 1,
        'nthread': 4,
        'silent': 1,
        'max_delta_step': 6
}
xgbm1.ini_params(params1)
# define the parameters of the xgboost and initialize them. i didn't fix the ini type..
name1 = 'grid_200_20_30_10_params_01_5_4_03_09_07_09_6_th7.csv'
xgbm1.train_and_predict_parallel(name1, upload_to_s3 = True)
# train and predict each grid individually and generate submission file with the name given,
# if upload_to_s3 is True, it zips the file and uploads the file to s3. for it to actually work, you have to update
# aws secret keys and access key in aws_config.py to yours.
# When i started playing with xgboost, I came to a conclusion that I need to do my runs on ec2 instead of my local machine
# as it is not very powerful. The above script will start training the data at 8 rows at a time, and in each row each grid
# cell will be trained serially with nthread = 4, so if you launch a c4.8xlarge machine all cpus will be used, with sane 
# load avgs.

In [ ]:
# Above code should work and should be able to generate a submission
# Here I will go into the details of the whole implementation. you can find the actual working
# code in the files mentioned above in the first lines..
# above two code blocks are implemented this way.

import time
import xgboost as xgb
from grid_generation import Grid, generate_grid_wise_cardinality_and_training_files
from grid_generation import get_top_3_places_of_dict, get_grids
from helpers import days, hours, quarter_days
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import train_test_split
from helpers import apk, zip_file_and_upload_to_s3

from base_scikit_learn_model import SklearnModel, get_grids_of_a_point

import numpy as np
import os
import pickle
import pdb

from multiprocessing import Pool
import logging
logging.basicConfig(filename='xgb_time_feature_runs.log', level=logging.DEBUG)

def map3eval(preds, dtrain):
    """
    custom evaluation function to be passed along with xgboost
    """
    actual = dtrain.get_label()
    predicted = preds.argsort(axis=1)[:,-np.arange(1,4)]
    metric = 0.
    for i in range(3):
        metric += np.sum(actual==predicted[:,i])/(i+1)
    metric /= actual.shape[0]
    return 'MAP@3', metric

class StateLoader(object):
    """
    class that helps in parallelising the code,
    this is because of python 2.7 limitation when it comes to
    using Pool.map
    
    using this we can just do something like 
    p = Pool(8)
    results = p.map(StateLoader(state), [1..m])
    p.close()
    p.join()
    """
    def __init__(self, state):
        self.state = state
    def __call__(self, row):
        return train_row(row, self.state)
"""
State:
    grid
    cv_grid
    test_grid
state = {
    'grid': Grid, the grid object
    'cv_grid': cv data converted to the grid specs, state['cv_data'][m][n] has the cv data that belong m,n grid
    'test_grid': same with test
    'threshold': 5
    'params_dict': xgb params for each grid
    'folder': {for giggles, i mean to save files} ignore this
"""


def train_row(i, state):
    """
    given the row number and the state, that has the grid object, cv/test data grid wise
    it will return test predictions, cv predictions and etc.
    """
    print "processing row %s" %(i)
    init_row_time = time.time()
    test_preds = []
    cv_preds = []
    train_preds = []
    for n in range(state['grid'].max_n + 1):
    # for n in range(1):
        if n % 10 == 0:
            print "processing column %s of row %s" %(n, i)
        clf, x_transformer, y_transformer, top_t = train_single_grid_cell(i, n, state)
        if top_t_train_places != None:
            train_preds.append(top_t_train_places)
        if len(state['test_grid'][i][n]) > 0:
            test_preds.append(predict_single_grid_cell(state['test_grid'][i][n], \
                clf, x_transformer, y_transformer, top_t, i, n, test = True, \
                ff = state['grid'].getFeaturesFolder(state['feature_submission_name'])))
        if len(state['cv_grid'][i][n]) > 0:
            cv_preds.append(predict_single_grid_cell(state['cv_grid'][i][n], \
                clf, x_transformer, y_transformer, top_t, i, n))
        del(clf)

    if len(test_preds) > 0:
        test_row = np.vstack(test_preds)
    else:
        test_row = None
    if len(cv_preds) > 0:
        cv_row = np.vstack(cv_preds)
    else:
        cv_row = None
    print "time taken for row %s is %s" %(i, time.time() - init_row_time)
    return (test_row, cv_row)

def train_single_grid_cell(m, n, state):
    # print m, n
    t = 10
    folder = state['folder']
    data = np.loadtxt(state['grid'].getGridFile(m, n), dtype = float, delimiter = ',')
    top_t = sorted(state['grid'].M[m][n].items(), cmp = lambda x, y: cmp(x[1], y[1]), reverse = True)[:t]
    top_t = map(lambda x: x[0], top_t)
    y = len(top_t)
    if y < t:
        top_t += [123]*(t-y) #[5348440074]*(t-y)
    if len(data) == 0:
        return None, None, None, top_t
    if len(data.shape) == 1:
        return None, None, None, top_t
    mask = np.array(map(lambda x: state['grid'].M[m][n][x] > state['threshold'], data[:, 5]))
    masked_data = data[mask, :]
    if len(masked_data) < 10:
        return None, None, None, top_t
    X, x_transformer = trans_x(masked_data[:, (1, 2, 3, 4)])
    Y, y_transformer = trans_y(masked_data[:, 5])

    if len(Y) == 0:
        return None, None, None, top_t, top_t_train_preds
    else:
        params = dict(state['params_dict'][m][n])
        params['num_class'] = len(y_transformer['encoder'].classes_)
        bst = classifier(X, Y, params)
        return bst, x_transformer, y_transformer, top_t
    pass

def predict_single_grid_cell(X, clf, x_transformer, y_transformer, top_t):
    t = 10
    data = np.array(X)
    if clf == None:
        top_t_placeids = np.array([top_t]*len(data))
    else:
        temp_x = trans_x(data[:, (1, 2, 3, 4)], x_transformer)[0]
        dtest = xgb.DMatrix(temp_x)
        prediction_probs = clf.predict(dtest)
        if len(prediction_probs.shape) == 1:
            prediction_probs = prediction_probs.reshape(-1, 1)
        top_t_placeids = y_transformer['encoder'].inverse_transform(np.argsort(prediction_probs, axis = 1)[:, ::-1][:, :t])
        x, y = top_t_placeids.shape
        if y < t:
            temp_array = np.array([top_t[:(t-y)]]*len(top_t_placeids))
            top_t_placeids = np.hstack((top_t_placeids, temp_array))
    return np.hstack((data[:, 0].reshape(-1, 1), top_t_placeids))


def trans_x(X, x_transformer = None):
    """
    X = [[x, y, a, t]] nx4 array and each row contais x, y, a, t
    x_transformer object contains additional information that are used to transform test data
    when its None, you should generate it and return it along with the scaled data. In this case transforming
    test data doenst need any extra. It will be useful when you use something like StandardSclaler, you need the
    encoder to transform test data
    """
    minute_v = X[:, 3]%60
    hour_v = X[:, 3]//60
    weekday_v = hour_v//24
    month_v = weekday_v//30
    year_v = (weekday_v//365 + 1)
    hour_v = ((hour_v%24 + 1) + minute_v/60.0)
    hour_v_2 = (X[:, 3]%(60*60*24))//(60*60*2)
    hour_v_3 = (X[:, 3]%(60*60*24))//(60*60*3)
    hour_v_4 = (X[:, 3]%(60*60*24))//(60*60*4)
    hour_v_6 = (X[:, 3]%(60*60*24))//(60*60*6)
    hour_v_8 = (X[:, 3]%(60*60*24))//(60*60*8)
    weekday_v = (weekday_v%7 + 1)
    month_v = (month_v%12 +1)
    accuracy_v = np.log10(X[:, 2])

    x_v = X[:, 0]
    y_v = X[:, 1]
    X_new = np.hstack((x_v.reshape(-1, 1),\
                     y_v.reshape(-1, 1),\
                     accuracy_v.reshape(-1, 1),\
                     hour_v.reshape(-1, 1),\
                     hour_v_2.reshape(-1, 1),\
                     hour_v_3.reshape(-1, 1),\
                     hour_v_4.reshape(-1, 1),\
                     hour_v_6.reshape(-1, 1),\
                     hour_v_8.reshape(-1, 1),\
                     weekday_v.reshape(-1, 1),\
                     month_v.reshape(-1, 1),\
                     year_v.reshape(-1, 1)))
    return (X_new, x_transformer)

def trans_y(y, y_transformer = None):
    """
    transform y to proper encoding and return y and its encoder
    """
    y = y.astype(int)
    if y_transformer == None:
        label_encoder = LabelEncoder()
        label_encoder.fit(y)
        y_transformer = {'encoder': label_encoder}
    new_y = y_transformer['encoder'].transform(y).reshape(-1, 1)
    return (new_y, y_transformer)


class XGB_Model(SklearnModel):

    def ini_params(self, params = default_xgb_params):
        self.params = params
        
    def train_and_predict_parallel(self, submission_file, upload_to_s3 = False):
        logging.debug('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
        init_time = time.time()
        
        # loading cv and test data into files
        cv_data = np.loadtxt(self.cross_validation_file, dtype = float, delimiter = ',')
        test_data = np.loadtxt(self.test_file, dtype = float, delimiter = ',')
        
        
        test_grid_wise_data = [[[] for n in range(self.grid.max_n + 1)]\
            for m in range(self.grid.max_m + 1)]
        cv_grid_wise_data = [[[] for n in range(self.grid.max_n + 1)]\
            for m in range(self.grid.max_m + 1)]

        print "converting test data to grid wise"
        for i in range(len(test_data)):
            m, n = get_grids_of_a_point((test_data[i][1], test_data[i][2]), self.grid)[0]
            test_grid_wise_data[m][n].append(test_data[i])

        print "converting cv data to grid wise"
        for i in range(len(cv_data)):
            m, n = get_grids_of_a_point((cv_data[i][1], cv_data[i][2]), self.grid)[0]
            cv_grid_wise_data[m][n].append(cv_data[i])

        state = {}
        state['grid'] = self.grid
        state['cv_grid'] = cv_grid_wise_data
        state['test_grid'] = test_grid_wise_data
        state['threshold'] = self.threshold

        xgb_params = self.params

        paramsFile = None # '../correct_row_wise_grid_search_results.pickle'# self.grid.getParamsFile(5, 123340)
        if paramsFile == None:
            print "params file doesn't exist.. so loading default params"
            state['params_dict'] = [[xgb_params for n in range(self.grid.max_n + 1)]\
                                        for m in range(self.grid.max_m + 1)]
        else:
            state['params_dict'] = pickle.load(open(paramsFile, 'rb'))

        submission_name = os.path.basename(submission_file)[:-4]


        p = Pool(8)
        row_results = p.map(StateLoader(state), range(self.grid.max_m + 1))
        p.close()
        p.join()
        del(test_grid_wise_data)
        del(cv_grid_wise_data)
        print "Training time of parallel processing %s" %(time.time() - init_time)
        logging.debug("Training time of parallel processing %s" %(time.time() - init_time))
        
        test_rows = map(lambda x: x[0], row_results)
        test_rows = filter(lambda x: x != None, test_rows)
        test_preds = np.vstack(test_rows).astype(int)
        sorted_test = test_preds[test_preds[:, 0].argsort()]
        
        cv_rows = filter(lambda x: x != None, cv_rows)
        cv_rows = map(lambda x: x[1], row_results)
        cv_preds = np.vstack(cv_rows).astype(int)
        sorted_cv = cv_preds[cv_preds[:, 0].argsort()]
        actual_cv = cv_data[:, -1].astype(int).reshape(-1, 1)
        cv_a_p = np.hstack((sorted_cv, actual_cv))
        
        apk_list = map(lambda row: apk(row[-1:], row[1:4]), cv_a_p)
        self.cv_mean_precision = np.mean(apk_list)
        print "mean precision of cross validation set", str(self.cv_mean_precision)
        logging.debug("mean precision of cross validation set: " + str(self.cv_mean_precision))
        logging.debug(submission_file)
        logging.debug("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")
        
        sorted_test = sorted_test.astype(str)
        submission = open(submission_file, 'wb')
        submission.write('row_id,place_id\n')
        for i in range(len(sorted_test)):
            row = sorted_test[i]
            row_id = row[0]
            row_prediction_string = ' '.join(row[1:4])
            submission.write(row_id + ',' + row_prediction_string + '\n')
            if i % 1000000 == 0:
                print "generating %s row of test data" %(i)
        submission.close()
        if upload_to_s3:
            zip_file_and_upload_to_s3(submission_file)

In [ ]:
# param search for xgboost
# I also had to impliment grid search for xgboost.train, cause I don't think sklearn wrapper is really complete

# all you have to do is run
# > python param_search.py 250 50 20 5 21 196 7
# the above arguments looks for the best params in the grid defn 250, 50, 20, 5.. and the next two args tells what
# grid cell to do the param search on, and the last argument defines the threshold, which eliminates place ids with 
# lesser threshold than that.

# it also runs 8 jobs in parallel, along with each xgboost with 4 threads

# if the files related to that above grid defn doesnt exist, it will generate and then does the performs the params search

g = grid.Grid(250, 50, 20, 5, pref = 'grid', files_flag = True)
g.generateCardinalityMatrix()

m = 12 # row no
n = 40 # column no
th = 7 # threshold

def transform_x(X, x_transformer = None):
    """
    X = [[x, y, a, t]]
    """
    minute_v = X[:, 3]%60
    hour_v = X[:, 3]//60
    weekday_v = hour_v//24
    month_v = weekday_v//30
    year_v = (weekday_v//365 + 1)
    hour_v = ((hour_v%24 + 1) + minute_v/60.0)
    hour_v_2 = (X[:, 3]%(60*60*24))//(60*60*2)
    hour_v_3 = (X[:, 3]%(60*60*24))//(60*60*3)
    hour_v_4 = (X[:, 3]%(60*60*24))//(60*60*4)
    hour_v_6 = (X[:, 3]%(60*60*24))//(60*60*6)
    hour_v_8 = (X[:, 3]%(60*60*24))//(60*60*8)
    weekday_v = (weekday_v%7 + 1)
    month_v = (month_v%12 +1)
    accuracy_v = np.log10(X[:, 2])
    x_v = X[:, 0]
    y_v = X[:, 1]
    return (np.hstack((x_v.reshape(-1, 1),
                     y_v.reshape(-1, 1),
                     accuracy_v.reshape(-1, 1),
                     hour_v.reshape(-1, 1),
                     hour_v_2.reshape(-1, 1),
                     hour_v_3.reshape(-1, 1),
                     hour_v_4.reshape(-1, 1),
                     hour_v_6.reshape(-1, 1),
                     hour_v_8.reshape(-1, 1),
                     weekday_v.reshape(-1, 1),
                     month_v.reshape(-1, 1),
                     year_v.reshape(-1, 1)), x_transformer)

def transform_y(y, y_transformer = None):
    """
    place_ids to encoded array
    """
    y = y.astype(int)
    if y_transformer == None:
        label_encoder = LabelEncoder()
        label_encoder.fit(y)
        y_transformer = {'encoder': label_encoder}
    new_y = y_transformer['encoder'].transform(y).reshape(-1, 1)
    return (new_y, y_transformer)

def map3eval(preds, dtrain):
    actual = dtrain.get_label()
    predicted = preds.argsort(axis=1)[:,-np.arange(1,4)]
    metric = 0.
    for i in range(3):
        metric += np.sum(actual==predicted[:,i])/(i+1)
    metric /= actual.shape[0]
    return 'MAP@3', metric

def load_data(m, n):
    f = g.getGridFile(m, n)
    return np.loadtxt(f, delimiter = ',')

def get_preds(probs, encoder):
    return encoder.inverse_transform(np.argsort(probs, axis = 1)[:, ::-1][:, :3])

def get_dtrain_enc(m, n):
    data = load_data(m, n)
    M = g.M
    mask = np.array(map(lambda x: M[m][n][x] > th, data[:, 5]))
    train = data[mask, :]
    print data.shape, "data_shape"
    X, x_tr = transform_x(train[:, (1, 2, 3, 4)])
    y, enc = transform_y(train[:, 5])
    print X.shape, "X shape"
    print y.shape, "y shape"
    print len(enc['encoder'].classes_), "no of classes"
    logging.debug("no of classes: %s" %(len(enc['encoder'].classes_)))

    dtrain = xgb.DMatrix(X, label=np.ravel(y))
    return (dtrain, enc)


tup = lambda t: list(itertools.izip(itertools.repeat(t[0]), t[1]))

def get_list_of_params(params_range):
    pr = list(map(tup, params_range.items()))
    pro = map(dict, list(itertools.product(*pr)))
    return pro

def grid_search_xgb(params_range_dict):
    grid_params_list = get_list_of_params(params_range_dict)
    p = Pool(8)
    maps = p.map(get_map_of_xgb, grid_params_list)
    p.close()
    p.join()
    sorted_maps = sorted(maps, cmp = lambda x, y: cmp(x['map'], y['map']), reverse = True)
    logging.debug("top map results")
    logging.debug(sorted_maps[:3])
    return sorted_maps

def get_map_of_xgb(grid_param):
    cv_params = dict(orig_params)
    num_class = {'num_class': len(enc['encoder'].classes_)}
    cv_params.update(num_class)
    cv_params.update(grid_param)
    # print orig_params, grid_param
    temp_cv = xgb.cv(cv_params, dtrain, num_boost_round = 100,
             early_stopping_rounds = 20, feval = map3eval, maximize = True)
    temp_map = temp_cv['test-MAP@3-mean'][temp_cv.shape[0]-1]
    grid_param['map'] = temp_map
    # print "cv results", grid_param
    return grid_param

#m, n = (12, 50) (37, 50) (12, 150) (37, 150)
#dtrain, enc = get_dtrain_enc(12, 50)
param_range1 = {
    'max_depth': range(2, 10, 1),
    'min_child_weight': range(1, 7, 1)
}

param_range2 = {
    'gamma': [i/10.0 for i in range(0, 6)],
    'colsample_bylevel': [i/10.0 for i in range(5, 10)]
}

param_range3 = {
    'max_delta_step': range(3, 15)
}

param_range4 = {
    'subsample': [i/100.0 for i in range(60, 100, 5)],
    'colsample_bytree': [i/100.0 for i in range(60, 100, 5)]
}




orig_params = {
    'nthread': 4,
    'eta': 0.1,
    'objective': 'multi:softprob',
    'max_depth': 8,
    'min_child_weight': 5,
    'gamma': 0.32,
    'subsample': 0.9,
    'colsample_bytree': 0.7,
    'scale_pos_weight': 1,
    'silent': 1
}
logging.debug("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")
dtrain, enc = get_dtrain_enc(m, n)
for param_range in [param_range1, param_range2, param_range3,\
    param_range4]:
    result = None
    try:
        result = grid_search_xgb(param_range)
    except Exception, e:
        print e
        print traceback.format_exc()
    if result != None:
        temp_param = result[0]
        del(temp_param['map'])
        orig_params.update(temp_param)
        logging.debug("best params so far")
        logging.debug(orig_params)

logging.debug("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
logging.debug("best params for the grid %s, %s, %s, %s" %(g.X, g.Y, g.xd, g.yd))
logging.debug("grid no m, n %s, %s" %(m, n))
logging.debug("threshold: %s" %(th))
logging.debug(orig_params)
logging.debug("###################################################")

In [ ]:
# miscellaneous functions

from launch_spot_instance import get_spot_prices, request_spot
# for this to work, you have to update sg, subnet, ami, key_name variables in the file, along with
# aws_config.py with your config keys

get_spot_prices(instance_type = 'c4.4xlarge')
# this will print the past hour spot price of the instance in sorted order in av zones
request_spot(price = '0.9', instance_type = 'm4.large', av_zone = 'us-east-1b', name = 'launched from notebook')
# this will request a spot instance.. and waits till the machine launches and prints the machines public ip




from helpers import zip_file_and_upload_to_s3

zip_file_and_upload_to_s3(file_name, bucket_name)
# zips the given file and uploads it to s3, aws_config should be updated for this to work